In [1]:
import pandas as pd
import requests
import time

In [4]:
# Cargar archivos
clients = pd.read_csv("data-caso1/clients.csv")
depots = pd.read_csv("data-caso1/depots.csv")

# Coordenadas
nodes = {}
nodes[0] = (depots.loc[0, "Longitude"], depots.loc[0, "Latitude"])  # Depósito

for idx, row in clients.iterrows():
    node_id = int(row["ClientID"])
    coord = (row["Longitude"], row["Latitude"])
    nodes[node_id] = coord

# Configuración de API
api_key = '5b3ce3597851110001cf6248be808c25e2964fb2b3e74f3c74d8ce63'
headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}
url = 'https://api.openrouteservice.org/v2/directions/driving-car'

# Crear grafo (matriz de distancias y tiempos)
grafo = {}

for i, coords_i in nodes.items():
    grafo[i] = {}
    for j, coords_j in nodes.items():
        if i == j:
            grafo[i][j] = {
                "distance_km": 999.0,
                "time_min": 999.0
            }
            continue

        params = {
            'start': f"{coords_i[0]},{coords_i[1]}",
            'end': f"{coords_j[0]},{coords_j[1]}"
        }

        print(f"Calculando de {i} a {j}...")
        response = requests.get(url, headers=headers, params=params)
        time.sleep(1.1)  # Evitar bloqueo por exceso de peticiones

        if response.status_code == 200:
            data = response.json()
            segmento = data['features'][0]['properties']['segments'][0]
            distancia = round(segmento['distance'] / 1000, 2)
            duracion = round(segmento['duration'] / 60, 2)
            grafo[i][j] = {
                "distance_km": distancia,
                "time_min": duracion
            }
        else:
            grafo[i][j] = {
                "distance_km": 999.0,
                "time_min": 999.0
            }
            print(f"Error {response.status_code} entre {i} y {j}")

print("Grafo generado correctamente")


Calculando de 0 a 1...


KeyboardInterrupt: 

In [6]:
import json

# Cargar el archivo JSON
with open('distancias-tiempo-api.json', 'r') as f:
    data = json.load(f)

# Crear la variable grafo
grafo = {}

for origen, destinos in data.items():
    grafo[int(origen)] = {}
    for destino, valores in destinos.items():
        distancia = valores['distance_km']
        tiempo = valores['time_min']
        if distancia == 0.0:
            distancia = 999.0
        if tiempo == 0.0:
            tiempo = 999.0
        grafo[int(origen)][int(destino)] = (distancia, tiempo)




In [7]:
print(grafo)

{0: {0: (999.0, 999.0), 1: (27.19, 37.94), 2: (16.99, 24.88), 3: (12.37, 21.25), 4: (26.7, 37.46), 5: (22.64, 37.66), 6: (19.65, 30.32), 7: (25.02, 41.42), 8: (23.89, 39.78), 9: (30.67, 45.27), 10: (33.86, 48.42), 11: (33.49, 45.98), 12: (15.87, 25.26), 13: (21.66, 32.79), 14: (10.36, 18.26), 15: (26.79, 37.02), 16: (33.91, 49.26), 17: (26.61, 40.59), 18: (19.87, 34.03), 19: (20.93, 29.42), 20: (13.14, 21.99), 21: (27.84, 38.78), 22: (24.41, 33.01), 23: (11.34, 19.85), 24: (30.87, 45.29)}, 1: {0: (30.96, 40.45), 1: (999.0, 999.0), 2: (14.29, 20.42), 3: (19.44, 24.53), 4: (1.13, 3.0), 5: (12.5, 18.04), 6: (11.32, 18.15), 7: (17.81, 34.15), 8: (13.75, 20.17), 9: (7.51, 11.23), 10: (7.39, 12.68), 11: (10.33, 11.93), 12: (25.81, 31.21), 13: (8.3, 14.06), 14: (20.31, 24.21), 15: (1.29, 2.81), 16: (7.44, 13.51), 17: (4.49, 10.82), 18: (9.84, 16.34), 19: (14.97, 23.16), 20: (23.09, 27.94), 21: (21.68, 31.86), 22: (5.38, 8.34), 23: (21.29, 25.8), 24: (4.89, 9.97)}, 2: {0: (18.34, 23.89), 1: (1

In [8]:
from pyomo.environ import (
    ConcreteModel, Set, Param, Var, Binary, NonNegativeReals,
    Objective, Constraint, minimize, SolverFactory
)
import pandas as pd

# Datos 
clients = pd.read_csv("data-caso1/clients.csv")
vehicles = pd.read_csv("data-caso1/vehicles.csv")

clientes = clients["ClientID"].tolist()
vehiculos = vehicles["VehicleID"].tolist()
nodos = [0] + clientes

demanda = dict(zip(clients["ClientID"], clients["Demand"]))
capacidad = dict(zip(vehicles["VehicleID"], vehicles["Capacity"]))


distancias = {(i,j): grafo[i][j][0] for i in nodos for j in nodos if i!=j}
tiempos    = {(i,j): grafo[i][j][1] for i in nodos for j in nodos if i!=j}

# Construcción del modelo 
model = ConcreteModel()

model.N = Set(initialize=nodos)
model.C = Set(initialize=clientes)
model.V = Set(initialize=vehiculos)

model.d = Param(model.C, initialize=demanda) # demanda de los clientes
model.q = Param(model.V, initialize=capacidad) # capacidad de los vehículos
model.dist = Param(model.N, model.N, initialize=distancias, default=1e6)
model.t = Param(model.N, model.N, initialize=tiempos, default=0)

# Costos 
model.Pf = Param(initialize=15000)   # precio combustible por litro
model.Cm = Param(initialize=700)     # costo mantenimiento por km
model.Ce = Param(initialize=0)       # costo eléctrico
model.Ft = Param(initialize=5000)    # tarifa fija por km
model.Cv = Param(initialize=0)       # costo por viaje

# δ_v = 1 para todos los vehículos (todos camionetas)
delta = { v: 1 for v in vehiculos }
model.delta = Param(model.V, initialize=delta)

# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)
model.u = Var(model.C, model.V, domain=NonNegativeReals)

# Función Objetivo
def obj_custom(m):
    return sum((m.Pf * m.dist[i,j] * m.delta[v] + m.Cm + m.delta[v] * m.Ce + m.Ft + m.Cv * m.t[i,j]) * m.x[i,j,v] for i in m.N for j in m.N for v in m.V if i!=j)
model.obj = Objective(rule=obj_custom, sense=minimize)



# Restricciones
def cliente_una_vez(m, j):
    return sum(m.x[i,j,v] for i in m.N for v in m.V if i!=j) == 1

def cliente_salida(m, j):
    return sum(m.x[j,k,v] for k in m.N for v in m.V if j!=k) == 1

def flujo_cliente(m, j, v):
    return (sum(m.x[i,j,v] for i in m.N if i!=j) - sum(m.x[j,k,v] for k in m.N if k!=j)) == 0

def mtz_rule(m, i, j, v):
    if i!=j:
        return m.u[i,v] + m.d[j] - m.u[j,v] <= m.q[v]*(1 - m.x[i,j,v])
    return Constraint.Skip

def limites_u(m, i, v):
    return (m.d[i], m.u[i,v], m.q[v])

def capacidad_total(m, v):
    return sum(m.d[j]*m.x[i,j,v]
               for i in m.N for j in m.C if i!=j) <= m.q[v]


model.cliente_una_vez = Constraint(model.C, rule=cliente_una_vez)
model.cliente_salida = Constraint(model.C, rule=cliente_salida)
model.flujo = Constraint(model.C, model.V, rule=flujo_cliente)
model.salida_dep = Constraint(model.V, rule=lambda m,v: sum(m.x[0,j,v] for j in m.C) <= 1)
model.entrada_dep = Constraint(model.V, rule=lambda m,v: sum(m.x[i,0,v] for i in m.C) <= 1)
model.mtz = Constraint(model.C, model.C, model.V, rule=mtz_rule)
model.lim_u = Constraint(model.C, model.V, rule=limites_u)
model.cap_total = Constraint(model.V, rule=capacidad_total)

# Solución del modelo
solver = SolverFactory("glpk")
solver.options["tmlim"]  = 60     # segundos máximo
solver.options["mipgap"] = 0.05   # gap 5%
results = solver.solve(model, tee=True)

print("Status:     ", results.solver.status)
print("Termination:", results.solver.termination_condition)
print("Obj. value: ", model.obj())

# Print resultados
print("\n RUTAS ASIGNADAS POR VEHÍCULO:")
for v in model.V:
    rutas, actual, visitados = [], 0, set()
    while True:
        nxt = next(
            (j for j in model.N if j!=actual and model.x[actual,j,v].value>0.5),
            None
        )
        if nxt is None or nxt in visitados:
            break
        rutas.append((actual, nxt))
        visitados.add(nxt)
        actual = nxt
    print(f" Vehículo {v}: {rutas or 'no asignado'}")




GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 60 --mipgap 0.05 --write C:\Users\57314\AppData\Local\Temp\tmp5z1ptty2.glpk.raw
 --wglp C:\Users\57314\AppData\Local\Temp\tmpzbudnfzu.glpk.glp --cpxlp C:\Users\57314\AppData\Local\Temp\tmp0d92w451.pyomo.lp
Reading problem data from 'C:\Users\57314\AppData\Local\Temp\tmp0d92w451.pyomo.lp'...
C:\Users\57314\AppData\Local\Temp\tmp0d92w451.pyomo.lp:62050: warning: lower bound of variable 'x2' redefined
C:\Users\57314\AppData\Local\Temp\tmp0d92w451.pyomo.lp:62050: warning: upper bound of variable 'x2' redefined
5064 rows, 4992 columns, 37056 non-zeros
4800 integer variables, all of which are binary
66850 lines were read
Writing problem data to 'C:\Users\57314\AppData\Local\Temp\tmpzbudnfzu.glpk.glp'...
56979 lines were written
GLPK Integer Optimizer 5.0
5064 rows, 4992 columns, 37056 non-zeros
4800 integer variables, all of which are binary
Preprocessing...
4680 rows, 4992 columns, 36672 non-zeros
4800 integ